In [1]:
from pprint import pprint
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import FastEmbedSparse
from qdrant_client.models import  SparseVector
from qdrant_client import QdrantClient, models
from langchain import PromptTemplate    
from langchain.schema import BaseMemory
from langchain.memory import ChatMessageHistory
import time
# from langchain.chains import RunnableWithMessageHistory
import numpy as np




In [2]:
import session_info
session_info.show()

C:\Users\quint\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\session_info\main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


In [2]:
# Load environment variables from .env file
load_dotenv()

# Access OpenAI API Key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is being correctly loaded
if openai_api_key is None or openai_api_key == "":
    print("OpenAI API Key not found! Please make sure it is set correctly in the .env file.")
else:
    print(f"OpenAI API Key Loaded: {openai_api_key[:4]}...")  


llmName = ChatOpenAI(model="gpt-4o")
Chat_llm = ChatOpenAI(model="gpt-4o", temperature=1)



dense_embedding_model = OpenAIEmbeddings(
                api_key=openai_api_key,
                model="text-embedding-3-large"
            )
sparse_embedding_model = FastEmbedSparse(model_name="Qdrant/bm25")

OpenAI API Key Loaded: sk-p...


Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

In [3]:
Clientpath = "VectorDB/CombinedData"
client = QdrantClient(path=Clientpath)
merged_collection_name = "merged vectors collection"

In [4]:
# Returns N dense nodes for the user question
def merged_dense_for_RRF(user_question, limitN):
    dense_query_vector =  dense_embedding_model.embed_query(user_question)
    merged_dense = client.query_points(
        merged_collection_name,
        query=dense_query_vector,
        using="text-embedding-3-large",
        limit=limitN,
        with_payload=True, 
    )

    return merged_dense

# Returns N sparse nodes for the user question
def merged_sparse_for_RRF(user_question, limitN):
    sparse_query_vector = sparse_embedding_model.embed_query(user_question)
    merged_sparse = client.query_points(
        collection_name=merged_collection_name,
        query=SparseVector(indices=sparse_query_vector.indices, values=sparse_query_vector.values),  
        using="bm25",
        with_payload=True,  
        limit=limitN  
    )
    
    return merged_sparse



In [21]:
# Combine results using Reciprocal Rank Fusion (RRF) 
def reciprocal_rank_fusion(results1, results2, k=60):
    """Combine results using Reciprocal Rank Fusion (RRF)."""
    rrf_scores = {}

    # Process results
    for rank, scored_point in enumerate(results1.points, start=1):  
        doc_id = scored_point.id
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + 1 / (k + rank)

    for rank, scored_point in enumerate(results2.points, start=1):  
        doc_id = scored_point.id
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + 1 / (k + rank)

    # Sort by the combined RRF score
    sorted_results = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)
    print(sorted_results)
    return sorted_results



In [1]:
# Grabs the payload of the fused results
def Fetch_chunks_back(fused_results):    
    fused_ids = [doc_id for doc_id, _ in fused_results]


    detailed_results = client.retrieve(
        collection_name=merged_collection_name,
        ids=fused_ids,
        with_payload=True,  # Include payload in the results
        with_vectors=False,  # Optional: Include vectors if needed
    )

    detailed_results_with_scores = []
    for result in detailed_results:
        # print(result)
        doc_id = result.id
        score = next((s for d, s in fused_results if d == doc_id), 0)
        detailed_results_with_scores.append({"ID":result.id, "rrf_score": score, "payload": result.payload} )
    return detailed_results_with_scores


In [ ]:
# old rephrase template not used for evaluation
costom_template = PromptTemplate(
    template="""
You are a highly knowledgeable assistant specializing in the JRC assessment framework (JRC code of conduct) and ISO standards (50001, 14001, 30134). Your task is to generate a developer-focused framework by interpreting relevant chunks from the provided content. Follow these strict instructions:

1. **Focus on Developer Needs**:
    - Provide actionable technical requirements for developers.
    - Ensure the requirements are framed in plain, developer-friendly language.

2. **Structure**:
    - Include clear headings for each requirement category.
    - Break down requirements into:
        - Title
        - Technical Details (if stated in chunk)
        - Evidence Required (if stated in chunk)
        - Reference Frameworks (if applicable) and Source (ISO/JRC section)
        - Type of requirement (mandatory, non-mandotory (optional), etc. for iso and WHole datacenter, new build, existing building, etc. and importance value for JRC)
        

3. **Comprehensiveness**:
    - Use **all relevant chunks** to create the framework, even if their relevance is marginal.
    -Mention which chunks you did not use and why.
    - Summarize overlapping or repetitive chunks to avoid redundancy but retain all key details.

4. **Avoid Assumptions**:
    - Only use information explicitly found in the chunks.
    - Do not introduce new concepts or requirements not in the source content.

5. **Formatting**:
    - Present the framework in a structured markdown format.
    - Use numbered sections and subsections for easy navigation.

6. **Metadata Inclusion**:
    - For each requirement, include references such as ISO/JRC standards, specific practice numbers, and evidence.

**Context**:
These chunks are sorted by relevance using the RRF method and pertain to the query.

**Chunks**:
{chunks}

**Question**:
{question}

**Answer**:
Provide a comprehensive developer-oriented framework meeting the above criteria.
"""
)


In [ ]:
# template for refining the awnser
Rephrase_question = PromptTemplate(template= """
You are tasked with rephrasing a user query for dense and sparse search on ISO and JRC documents. 
Exclude irrelevant parts of the question that are meant for LLM-based reasoning or conversational context. 
Focus on crafting a concise and precise query to retrieve relevant content from the documents, while enabling effective term expansion for hybrid search.

**User Query**:
{user_query}

**Instructions**:
1. Extract only the part of the query that is relevant for searching ISO or JRC documents.
2. Enhance the query with terms or synonyms that improve recall during search Make sure that the terms added are relevant to the original query, so storage only storage related terms:
3. Use document-specific terminology, synonyms, or relevant standards where applicable.

**Search Query**:                                 
Please provide a concise and precise search query based on the above instructions.                                   
                                   
         
                                   """)


def rephase_query(user_query):
   new_quest = Rephrase_question.format(user_query=user_query)
   new_query = Chat_llm.invoke(new_quest)
    
   return new_query
 
 

In [ ]:
# Pre-refined template not used for evaluation
refine_template = PromptTemplate(template="""
You are a highly knowledgeable assistant specializing in the JRC assessment framework (JRC code of conduct) and ISO standards (50001, 14001, 30134). 
Your task is to refine or improve or awnser the user query over the previous aswser provided by the LLM.

This is the prompt that the previous LLM was given


"Your task is to generate a developer-focused framework by interpreting relevant chunks from the provided content. Follow these strict instructions:

1. **Focus on Developer Needs**:
    - Provide actionable technical requirements for developers.
    - Ensure the requirements are framed in plain, developer-friendly language.

2. **Structure**:
    - Include clear headings for each requirement category.
    - Break down requirements into:
        - Title
        - Technical Details (if stated in chunk)
        - Evidence Required (if stated in chunk)
        - Reference Frameworks (if applicable) and Source (ISO/JRC section)
        - Type of requirement (mandatory, non-mandotory (optional), etc. for iso and WHole datacenter, new build, existing building, etc. and importance value for JRC)
        

3. **Comprehensiveness**:
    - Use **all relevant chunks** to create the framework, even if their relevance is marginal.
    -Mention which chunks you did not use and why.
    - Summarize overlapping or repetitive chunks to avoid redundancy but retain all key details.

4. **Avoid Assumptions**:
    - Only use information explicitly found in the chunks.
    - Do not introduce new concepts or requirements not in the source content.

5. **Formatting**:
    - Present the framework in a structured markdown format.
    - Use numbered sections and subsections for easy navigation.

6. **Metadata Inclusion**:
    - For each requirement, include references such as ISO/JRC standards, specific practice numbers, and evidence."



**Previous Query Chunks**:
{chunks}

**History**:
Conversation history, including prior LLM responses:
{history}

**User Question**:
{question}

**Answer**:
""")


In [ ]:
# Set up a class that combines the memory with retrieval capabilities
class MemoryWithRetrieval:
    def __init__(self):
        self.message_history = ChatMessageHistory()  # In-memory conversation storage

    def add_to_memory(self, user_input, bot_response):
        self.message_history.add_user_message(user_input)
        self.message_history.add_ai_message(bot_response)

    def get_memory_context(self, limit=5):
        # Retrieve the last `limit` messages for context
        messages = self.message_history.messages[-limit:]
        return "\n".join([f"{msg.type.capitalize()}: {msg.content}" for msg in messages])
    
    def reset_memory(self):
        self.message_history = ChatMessageHistory()  # Clear the conversation history

In [ ]:
# Fetch chunks and combine with RRF
def fetch_chunks_rrf_sorted(user_question, limitN):
    dense_results = merged_dense_for_RRF(user_question, limitN)
    print("dense chunks:", dense_results)
    sparse_results = merged_sparse_for_RRF(user_question, limitN)
    print("sparse chunks:", sparse_results)
    fused_results = reciprocal_rank_fusion(dense_results, sparse_results, k=60)
    Nodes_rrf_back = Fetch_chunks_back(fused_results)
    return Nodes_rrf_back



In [ ]:

# Initialize memory and LLM
memory = MemoryWithRetrieval()
previous_chunks = None

# for normal query
def conversational_system(query):
   
    # Fetch context from the fused results
    new_query = rephase_query(query)
    context = fetch_chunks_rrf_sorted(new_query.content, 15)
    print("Query used: ", new_query)
    
    # Get conversation memory context
    # memory_context = memory.get_memory_context(limit=7)
    
    # print("Current Conversation History:")
    # print(memory_context)
    
    
    # Generate response

    
    custom_prompt = costom_template.format(chunks=context[:20], question=query)
    response = Chat_llm.invoke(custom_prompt)
    
    # Add to memory
    memory.add_to_memory(query, response)
    global previous_chunks
    previous_chunks == context[:20]
    
    print(len(context[:15]))
    
    print("Your query: ", query)
    return response , context





In [ ]:
# for refining awnser absed on query
def refine_system(query):
    
    
    
    print("test")

    # Get conversation memory context
    memory_context = memory.get_memory_context(limit=10)
    
    print("### Current Conversation History ###")
    print(memory_context)
    
    refine_query = query.replace("refine", "").strip()

    
    # Generate response
    custom_prompt = refine_template.format(question=refine_query, history=memory_context, chunks = previous_chunks)
    response = Chat_llm.invoke(custom_prompt)
    
    # Add to memory
    memory.add_to_memory(query, response)
    
    # print("Your query: ", query)
    return response



In [ ]:
# Talk to the system
while True:
    chunks_used1 = []
    
    user_input = input("User: ")
    if user_input.lower() == "quit":
        print("Exiting the system.")
        break
    
    if user_input.lower() == "exit":
        print("Exiting the system and resetting memory.")
        memory.reset_memory()
        print("\n" * 10)
        break
    
    if user_input.lower() == "test":
        print("test")
        continue
        
    
    if user_input.lower() == "reset":
        memory.reset_memory()
        print("\n" * 10)
        print("Memory has been reset.")
        continue    
        
        
    
    start_time = time.time()  # Measure processing time
    bot_response, chunks_used = conversational_system(user_input)
    end_time = time.time()


    print(f"\nDebug: conversational_system took {end_time - start_time:.2f} seconds", flush=True)
    print("\n", f"Bot: {bot_response.content if bot_response else 'No Response'}", flush=True)
    print("\n", "Chunks used:", chunks_used, flush=True)
    
    
    # user_input = input("User: ")
    # if user_input.lower() in ["refine"]:
    #     bot_response = refine_system(user_input)
    #     break
    

    # bot_response, chunks_used = conversational_system(user_input)



dense chunks: points=[ScoredPoint(id='84970a49-031c-466e-a29b-3145e79d1ef3', version=0, score=0.5950744432926264, payload={'content': 'A.2 Data\tcentre\tavailability\tobjectives\nThe availability objectives of a data centre can restrict such improvements due to the use of redundant infrastructure components and infrastructures necessary to respond the following circumstances: a) the user of the data centre will either set or demand a series of service level agreements (SLAs) based upon the criticality of their business case covering continuity and quality of power and environmental control provision, connectivity and security; b) the SLAs will reflect directly the appetite for risk that the user has and their ability or willingness to accept shutdowns for maintenance and repairs in the critical systems; c) data centres whose users can accept high levels of risk, scheduled shutdowns for maintenance and shutdowns for fault repair can reach very low design KPIs; d) data centres whose user

: 